# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'traffic-signs-data/train.p'
validation_file='traffic-signs-data/valid.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    validation = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_val, y_val = validation['features'], validation['labels']
X_test, y_test = test['features'], test['labels']

load_and_enqueue -> queue -> dequeue_and_train

In [2]:
import tensorflow as tf
import threading
import time
import logging
import numpy as np

# Use logger for console output
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) #logging.INFO logging.DEBUG
if logger.handlers == []:
    ch = logging.StreamHandler()
    logger.addHandler(ch)

logger.info('TF version = {}'.format(tf.__version__))

# Prepare a small training set for testing multithread tensorflow
queue_capcity = 2

logger.debug('')
logger.debug('X_train {}'.format(X_train.shape))
logger.debug('y_train {}'.format(y_train.shape))
X_train = X_train[:queue_capcity]
y_train = y_train[:queue_capcity]
logger.debug('X_train {}'.format(X_train.shape))
logger.debug('y_train {}'.format(y_train.shape))

# Define a tensorflow Queue for data pre-processing
with tf.device('/cpu:0'):
    feature_input = tf.placeholder(tf.int32, shape=[32, 32, 3], name='feature_input')
    label_input = tf.placeholder(tf.int32, shape=[], name='label_input')
    queue = tf.FIFOQueue(queue_capcity, [tf.int32, tf.int32], shapes=[[32, 32, 3], []])

    # Define logic of load and enqueue
    enqueue_op = queue.enqueue([feature_input, label_input])

def load_and_enqueue(sess, coord, X_train, y_train, enqueue_op):
    i = 0
    while not coord.should_stop():
        time.sleep(3)
        
        # Load data and do pre-processing
        feature = X_train[i%queue_capcity]
        label = y_train[i%queue_capcity]
        
        # Enqueue training data        
        sess.run(enqueue_op, feed_dict = {feature_input: feature,
                                          label_input: label})
        logger.debug('')
        logger.debug('load_and_enqueue round {}'.format(i))
        logger.debug('Enqueue feature {}'.format(feature[0,0,:]))
        logger.debug('Enqueue label {}'.format(label))
        logger.debug('Enqueue queue size = {}'.format(queue.size().eval(session=sess)))
        logger.debug('')
        i += 1

# Define logic of dequeue and training
with tf.device('/cpu:0'):
    feature_dq_op, label_dq_op = queue.dequeue()

with tf.device('/gpu:0'):
    feature_new = tf.placeholder(tf.int32, shape=[32, 32, 3], name='feature_new')
    label_new = tf.placeholder(tf.int32, shape=[], name='label_new')
    weight = tf.constant(1, shape=[32, 32, 3])
    feature_output = tf.add(feature_new, weight, name='feature_add')
    
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    logger.debug('')
    logger.debug('Initial queue size = {}'.format(queue.size().eval(session=sess)))
    
    # Start data enqueue thread
    coord = tf.train.Coordinator()
    t = threading.Thread(target = load_and_enqueue,
                         args = (sess, coord, X_train, y_train, enqueue_op))
    t.start()
    
    # Dequeue feature and label
    for i in range(queue_capcity*2):
        time.sleep(6)
        feature, label = sess.run([feature_dq_op, label_dq_op])
        feature_final = sess.run([feature_output], feed_dict = {feature_new: feature,
                                                                label_new: label})
        feature_final = np.squeeze(feature_final)
        logger.debug('')
        logger.debug('Dequeue_and_train round {}'.format(i))
        logger.debug('Dequeue feature = {}'.format(feature[0,0,:]))
        logger.debug('Dequeue label = {}'.format(label))
        logger.debug('Dequeue queue size = {}'.format(queue.size().eval(session=sess)))
        logger.debug('Dequeue feature_final = {}'.format(feature_final[0,0,:]))
        logger.debug('')
        
    # Finish training and stop enqueue thread
    coord.request_stop()
    coord.join([t])
    logger.debug('')
    logger.debug('Enqueue thread stopped')
    logger.debug('Final queue size = {}'.format(queue.size().eval(session=sess)))
    logger.debug('')
    


TF version = 1.4.0

X_train (34799, 32, 32, 3)
y_train (34799,)
X_train (2, 32, 32, 3)
y_train (2,)

Initial queue size = 0

load_and_enqueue round 0
Enqueue feature [28 25 24]
Enqueue label 41
Enqueue queue size = 1


Dequeue_and_train round 0
Dequeue feature = [28 25 24]
Dequeue label = 41
Dequeue queue size = 0

load_and_enqueue round 1
Dequeue feature_final = [29 26 25]
Enqueue feature [28 24 24]

Enqueue label 41
Enqueue queue size = 1


load_and_enqueue round 2
Enqueue feature [28 25 24]
Enqueue label 41
Enqueue queue size = 2


Dequeue_and_train round 1
Dequeue feature = [28 24 24]
Dequeue label = 41
Dequeue queue size = 1
Dequeue feature_final = [29 25 25]


load_and_enqueue round 3
Enqueue feature [28 24 24]
Enqueue label 41
Enqueue queue size = 2



load_and_enqueue round 4
Dequeue_and_train round 2
Enqueue feature [28 25 24]
Dequeue feature = [28 25 24]
Enqueue label 41
Dequeue label = 41
Enqueue queue size = 2

Dequeue queue size = 2
Dequeue feature_final = [29 26 25]



De

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_val = X_val.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_val))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_val)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

### Show one traffic sign example of each class in training set

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

import csv

# Import traffic signs class names
sign_names=[]
with open('signnames.csv', 'rt') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        sign_names.append((row['SignName']))
sign_names = np.array(sign_names)

# Show one traffic sign example of each class
y_train_list = y_train.tolist()
num_row = 3
num_col = 15
size = 4.6

f, axarr = plt.subplots(num_col, num_row, figsize=(size * num_row, size * num_col))
counter = 0
for i in range(num_col):
    for j in range(num_row):
        counter = i * num_row + j
        if counter >= n_classes:
            break
        axarr[i, j].imshow(train['features'][y_train_list.index(counter)])
        axarr[i, j].set_title(str(counter) + ": "+ sign_names[counter])
        axarr[i, j].axis('off')

### Label class distribution in Training, Validation, and Test sets

In [ ]:
# Show label class distribution of traffic sign image in training set
x = np.unique(y_train)
y = np.bincount(y_train)

fig, ax = plt.subplots()
ax.plot(x, y)

ax.set(xlabel='label class', ylabel='number of image samples',
       title='training set label class distribution')
ax.grid()


# Show label class distribution of traffic sign image in validation set
x = np.unique(y_val)
y = np.bincount(y_val)

fig, ax = plt.subplots()
ax.plot(x, y)

ax.set(xlabel='label class', ylabel='number of image samples',
       title='validation set label class distribution')
ax.grid()

# Show label class distribution of traffic sign image in test set
x = np.unique(y_test)
y = np.bincount(y_test)

fig, ax = plt.subplots()
ax.plot(x, y)

ax.set(xlabel='label class', ylabel='number of image samples',
       title='test set label class distribution')
ax.grid()

plt.show()


## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

import tensorflow as tf

print('tensorflow version = {}'.format(tf.__version__))

# Use training set mean and standard deviation to normalize train/val/test set
def normalize(X, mean, std):
    #X_norm = (X - mean) / std
    return X

X_train_mean = np.mean(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)

X_train = normalize(X_train, X_train_mean, X_train_std)
X_val = normalize(X_val, X_train_mean, X_train_std)
X_test = normalize(X_test, X_train_mean, X_train_std)

### Data augmentation
* Use PCA to do color augmentation (idea is from AlexNet paper: https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)

In [ ]:
def PCA_preprocessing(image):
    # Normalize RGB of each pixel by substracting the mean RGB of this image
    image_flatten = np.reshape(image, (1024, -1))
    image_mean = np.mean(image_flatten, axis=0)
    image_flatten_norm = image_flatten - image_mean

    # Use SVD to compute eigenvectors and eigenvalues of RGB covariance matrix
    u, s, _ = np.linalg.svd(image_flatten_norm.T)
    return u, s, image_mean


def PCA_augmentation(image, u, s, image_mean):
    # Do PCA color augmentation
    image_delta = np.dot(u, np.transpose(s**2 / 1024 * np.random.normal(loc=0.0, scale=0.1, size=1)))
    image_final = image - image_mean + image_delta
    return image_final

# Test PCA color augmentation 
image = X_train[5500]  #train['features'][5501]
u, s, image_mean = PCA_preprocessing(image)
image_final = PCA_augmentation(image, u, s, image_mean)
plt.imshow(image)
plt.show()
plt.imshow(image_final)
plt.show()

### Use multiple threads to do PCA pre-processing for training set images

In [ ]:
import threading
import time
import queue as q
import logging
import sys

# Use logger for console output
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) #logging.INFO logging.DEBUG
if logger.handlers == []:
    ch = logging.StreamHandler()
    logger.addHandler(ch)

# Create a queue to exchange info between main process and worker threads
queue = q.Queue()

# Define pca preprocessing thread by inheriting threading.Thread
class pca_preprocessing_thread (threading.Thread):
    '''
    Thread for running pca pre-processing
    '''  
    def __init__(self, threadID, name, start_index, end_index, train_set, queue):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.start_index = start_index
        self.end_index = end_index
        self.train_set = train_set
        self.queue = queue
        self.u_batch = []
        self.s_batch = []
        self.image_mean_batch = []
   
    def run(self):
        '''
        main execution method of pca thread
        '''
        logger.info("Starting " + self.name)
        self.pca_preprocessing_batch()
        logger.info("Exiting " + self.name)

    def pca_preprocessing_batch(self):
        """
        Compute RGB eigenvectors and eigenvalues for each image in this batch 
        and store the result in the queue for further aggregation in the main process
        """
        logger.debug("")
        logger.debug('{} started'.format(self.name))
        logger.debug("")
        
        try:
            for i in range(self.start_index, self.end_index):
                if i % 100 == 0:
                    logger.debug('{}, current_image_index={}'.format(self.name, i))
                    logger.debug("")
                u, s, image_mean = PCA_preprocessing(self.train_set[i])
                self.u_batch.append(u)  # RGB eigenvectors
                self.s_batch.append(s)  # Square root of RGB eigenvalues
                self.image_mean_batch.append(image_mean)
        except Exception as e:
            logger.error('Got exception in {}'.format(self.name))
            logger.error('Error: ' + str(e))

        logger.debug('{} started to put result to queue'.format(self.name))
        self.queue.put([self.start_index, self.end_index, self.u_batch, self.s_batch, self.image_mean_batch])
        logger.debug('{} finished'.format(self.name))
        logger.debug("")        
        

# Create new threads
batch_size = 4800
thread_pool = []

for offset in range(30000, 34799, batch_size):
    end = min(offset + batch_size, n_train)
    thread = pca_preprocessing_thread(offset, "Thread-"+str(offset), 
                                      start_index=offset, end_index=end, 
                                      train_set=X_train, queue=queue)
    thread_pool.append(thread)

start = time.time()

# Start PCA pre-processing threads
for t in thread_pool:
    t.start()

# Join these threads so that the main process is blocked until all of the threads are finished
for t in thread_pool:
    t.join()

# Aggregate results from all PCA pre-processing threads
start_index_train = []
end_index_train = []
u_train = []
s_train = []
image_mean_train = []

for i in range(len(thread_pool)):
    start_index, end_index, u_batch, s_batch, image_mean_batch = queue.get()
    start_index_train.append(start_index)
    end_index_train.append(end_index)
    u_train += u_batch
    s_train += s_batch
    image_mean_train += image_mean_batch
    
logger.debug('start_index_train {}'.format(start_index_train))    
logger.debug('end_index_train {}'.format(end_index_train))    
logger.info('u_train {}'.format(np.shape(u_train)))
logger.info('s_train {}'.format(np.shape(s_train)))
logger.info('image_mean_train {}'.format(np.shape(image_mean_train)))

end = time.time()
logger.info('PCA augmentation pre-processing takes {:.3f} seconds'.format(end - start))

### LeNet Model Architecture

* Input [32x32x3]
* 6 Filters, 5x5, Stride 1, ReLU [28x28x6]
* Batch Normalization
* Max Pool 2x2, Stride 2 [14x14x6]
* 16 Filters, 5x5, Stride 1, ReLU [10x10x16]
* Batch Normalization
* Max Pool 2x2, Stride 2 [5x5x16]
* Flatten [400]
* Fully Connected size 120, ReLU [120]
* Dropout
* Fully Connected size [n_classes = 43]


In [ ]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    #with tf.name_scope('stddev'):
    #  stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    #tf.summary.scalar('stddev', stddev)
    #tf.summary.scalar('max', tf.reduce_max(var))
    #tf.summary.scalar('min', tf.reduce_min(var))
    #tf.summary.histogram('histogram', var)
    

def LeNet(x, regularizer, is_training, dropout_rate):
    """
    Compute LeNet class score for a batch of input images.
    
    Inputs:
    - x: placeholder of input images, shape [batch_size, 32, 32, 3]
    - regularizer: L2 regularizer operator
    - is_training: bool placeholder to indicate batch normalization is in training mode (True) or inference mode (False)
    - dropout_rate: placeholder to specify the dropout rate
    
    Returns:
    TensorFlow Tensor with shape [batch_size, n_classes], 
    containing the class score for each input image.
    """
    global n_classes
    
    with tf.name_scope("LeNet"):
        # Use xavier initializer as initializer for all weight variables
        init = tf.contrib.layers.xavier_initializer(uniform=True)

        # CONV -> BN -> POOL
        a = tf.layers.conv2d(inputs=x, filters=6, kernel_size=5, padding='VALID', activation=tf.nn.relu,
                             kernel_initializer=init, kernel_regularizer=regularizer, name='conv2d_1')
        variable_summaries(a) # check BN result in TensorBoard
        a = tf.layers.batch_normalization(inputs=a, training=is_training, name='bn_1')
        variable_summaries(a)
        a = tf.layers.max_pooling2d(a, pool_size=2, strides=2, padding='SAME', name='maxpool_1')

        # CONV -> BN -> POOL
        a = tf.layers.conv2d(inputs=a, filters=16, kernel_size=5, padding='VALID', activation=tf.nn.relu,
                             kernel_initializer=init, kernel_regularizer=regularizer, name='conv2d_2')
        a = tf.layers.batch_normalization(inputs=a, training=is_training, name='bn_2')
        a = tf.layers.max_pooling2d(a, pool_size=2, strides=2, padding='SAME', name='maxpool_2')

        # FC -> Dropout -> FC -> Logits
        a = tf.contrib.layers.flatten(a)
        a = tf.layers.dense(a, 120, activation=tf.nn.relu, kernel_initializer=init, 
                            kernel_regularizer=regularizer, name='dense_1')
        a = tf.layers.dropout(a, rate=dropout_rate, training=is_training, name='dropout')
        logits = tf.layers.dense(a, n_classes, kernel_initializer=init, 
                                 kernel_regularizer=regularizer, name='logits')

    return logits

### Load and enqueue training data
* Use CPU thread to run this function to prepare augmented training data for training process on GPU

In [ ]:
# Define data augmentation thread by inheriting threading.Thread
class data_augmentation_thread (threading.Thread):
    '''
    Thread for running PCA data augmentation
    '''  
    def __init__(self, threadID, name, sess, batch_size, X_train, y_train, queue):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.sess = sess
        self.batch_size = batch_size
        self.X_train = X_train
        self.y_train = y_train
        self.queue = queue
        self.n_train = 4700
        self.stoprequest = threading.Event()

    def join(self, timeout=None):
        '''
        Main process can terminate the thead by calling join() method
        '''
        self.stoprequest.set()
        super(data_augment_thread, self).join(timeout)
        
    def run(self):
        '''
        main execution method of thread
        '''
        logger.info("Starting " + self.name)
        self.load_and_enqueue()
        logger.info("Exiting " + self.name)

    def load_and_enqueue(self):
        """
        load original training image, do PCA color augmentation, and enqueue the augmented image
        """
        logger.debug('{} started'.format(self.name))
        
        # Create a sub-graph for data augmentation
        with tf.device('/cpu:0'):
            X_batch_input = tf.placeholder(tf.uint8, shape=[None, 32, 32, 3])
            y_batch_input = tf.placeholder(tf.uint8, shape=[None])
            enqueue_op = self.queue.enqueue_many([X_batch_input, y_batch_input])
        
        while not self.stoprequest.isSet():
            try:
                for offset in range(0, self.n_train, self.batch_size):
                    end = min(offset + self.batch_size, self.n_train)
                    batch_X = self.X_train[offset, end]
                    batch_y = self.y_train[offset, end]
                    for i in range(offset, end):
                        batch_X[i] = PCA_augmentation(batch_X[i], u_train[i], s_train[i], image_mean_train[i])
                    self.sess.run(enqueue_op, feed_dict={X_batch_input: batch_X,
                                                         y_batch_input: batch_y})      
            except Queue.Empty:
                continue
            except Exception as e:
                logger.error('Got exception in {}'.format(self.name))
                logger.error('Error: ' + str(e))
            
        logger.debug('{} finished'.format(self.name))

### Model evaluation and training

In [ ]:
from sklearn.utils import shuffle

def evaluate(X_data, y_data, accuracy, dropout_rate_value=0.4, batch_size=64):
    '''
    Evaluate predict accuracy on a data set, normally validation set
    
    Input:
    - X_data: data set for placeholder x
    - y_data: label set for placeholder y
    - accuracy: operator of prediction accuracy
    - dropout_rate_value: value of dropout rate
    - batch_size: mini-batch size for evaluation
    
    Return:
    - accuracy: total prediction accuracy on this data set
    '''
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        batch_accuracy = sess.run(accuracy,
                                  feed_dict={x: batch_x, 
                                             y: batch_y, 
                                             is_training: False,
                                             dropout_rate: dropout_rate_value})
        total_accuracy += (batch_accuracy * len(batch_x))
    return total_accuracy / num_examples


def train(sess, loss, accuracy, train_step, dropout_rate_value, X_train, y_train, X_val, y_val,
          batch_size=64, num_epoch=10, print_every=100, verbose=False):
    '''
    Run mini-batch training, show training loss, training accuracy, and validation accuracy

    Input:
    - sess: session to run model training
    - loss: operator of model loss
    - accuracy: operator of prediction accuracy
    - train_step: operator of training step to minimize loss
    - dropout_rate_value: value of dropout rate
    - X_train: training set data for placeholder x
    - y_train: training set label for place holder y
    - X_val: validation set data for placeholder x
    - y_val: validation set data for placeholder y
    - batch_size: mini-batch size of training data
    - num_epoch: number of training epochs
    - print_every: number of iterations to print training loss and accuracy
    - verbose: print detailed info when it set to True

    Returns:
    - train_accuracy: prediction accuracy on last epoch of training
    - validation_accuracy: prediction accuracy on validation set
    '''    
    
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)        
    
    train_accuracy_history = []
    val_accuracy_history = []
    loss_history = []
    
    # Define TensorBoard FileWriter for tensor visualization
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./' + model + '/train',sess.graph)
    
    # Define variables for sess.run()
    variables = [loss, accuracy, train_step, merged]
    
    if verbose:
        print("")
        print("Training...")
    
    # Start a CPU thread to do data augmentation, and enqueue augmented data
    queue = tf.FIFOQueue(256, [tf.uint8, tf.uint8], shapes=[[32, 32, 3], []])
    data_aug_thread = data_augmentation_thread(0, "Data-augmentation-thread", 
                                               sess, batch_size, X_train, y_train, queue)
    data_aug_thread.start()
    
    # Train model in GPU
    for i in range(num_epoch):
        epoch_correction = 0.0
        
        for offset in range(0, n_train, batch_size):
            # Get mini batch of training data from Queue
            batch_x, batch_y = queue.dequeue_many(batch_size)
            logger.debug('batch_x {}'.format(np.shape(batch_x)))
            logger.debug(batch_x)
            logger.debug('batch_y = {}'.format(np.shape(batch_y)))
            logger.debug(batch_y)

            # Train model with mini batch of training data
            batch_loss, batch_accuracy, \
            _, summary = sess.run(variables, 
                                  feed_dict={x: batch_x, 
                                             y: batch_y, 
                                             is_training: True,
                                             dropout_rate: dropout_rate_value})

            iter_counter = offset // batch_size
            
            # Add tensor summary to TensorBoard to visualize
            #train_writer.add_summary(summary, iter_counter)  
            
            # Add batch loss to loss history
            loss_history.append(batch_loss)
            
            # Aggreate batch correct predictions to epoch level
            epoch_correction += batch_accuracy * len(batch_x)
        
            # Print loss and batch training accuracy after every "print_every" iterations            
            if verbose and iter_counter % print_every == 0:
                print("iteration:{}, batch loss={:.3f}, batch accuracy={:.3f}".
                      format(iter_counter, batch_loss, batch_accuracy))
                    
        # Add epoch training accuracy and epoch validation accuracy to accuracy history
        train_accuracy = epoch_correction / num_examples
        validation_accuracy = evaluate(X_val, y_val, accuracy, dropout_rate_value, batch_size)
        train_accuracy_history.append(train_accuracy)
        val_accuracy_history.append(validation_accuracy)
        
        # Print accuracy
        if verbose:
            print("EPOCH:{}, train_accuracy={:.3f}, validation accuracy={:.3f}".
                  format(i+1, train_accuracy, validation_accuracy))

    data_aug_thread.join()
    
    return train_accuracy, validation_accuracy, \
           loss_history, train_accuracy_history, val_accuracy_history

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

### Train model

In [ ]:
model = 'LeNet'

# Hyperparameters candidate values
learning_rates = 10 ** np.random.uniform(-3, -2.3, 1)           # 1e-03 ~ 5e-03
regularization_scales = 10 ** np.random.uniform(-2, -1.3, 1)    # 1e-03 ~ 1e-01
dropout_rates = [0.7]    # 0.6 ~ 0.8
batch_sizes = [128]      # 64, 128
num_epoch = 20           # 10
decay_rate = 0.95        # 0.9 ~ 0.95

# Dictionary to store accuracy result of hyperparameter combinations
hyperparameter_dict = {}

# Variable to store the best_val_accuracy so far, which is also an indicator of saving best model
best_val_accuracy = 0.0

# Search for best hyperparameter combination
print('model={}'.format(model))
iteration_counter = 0
for lr in learning_rates:
    for rs in regularization_scales:
        for dr in dropout_rates:
            for bs in batch_sizes:
                # Reset tf graph for each iteration 
                tf.reset_default_graph()

                with tf.device('/gpu:0'):
                    # Define place holders
                    x = tf.placeholder(tf.float32, (None, 32, 32, 3), name='x')
                    y = tf.placeholder(tf.int32, (None), name='y')
                    is_training = tf.placeholder(tf.bool, name='is_training')
                    dropout_rate = tf.placeholder(tf.float32, name='dropout_rate')

                    # Compute logits with certain regularization scale
                    regularizer = tf.contrib.layers.l2_regularizer(rs)
                    logits = LeNet(x, regularizer, is_training, dropout_rate)

                    # Define accuracy
                    one_hot_y = tf.one_hot(y, n_classes, name='one_hot_y')        
                    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1), 
                                                  name='correct_prediction')
                    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), 
                                              name='accuracy')

                    # Define loss with L2 regularization
                    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits,
                                                                            name='cross_entropy')
                    loss = tf.reduce_mean(cross_entropy, name='loss')
                    loss += tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES), name='L2_Reg')

                    # Define train step with optimizer and decayed learning rate
                    global_step = tf.Variable(0, trainable=False)
                    learning_rate = tf.train.exponential_decay(lr, global_step, n_train / bs, 
                                                               decay_rate, staircase=True, 
                                                               name='exp_decayed_learning_rate')
                    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam_Optimizer')
                    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
                    with tf.control_dependencies(extra_update_ops):
                        train_step = optimizer.minimize(loss, global_step=global_step, name='train_step')

                # Define saver to save best model
                saver = tf.train.Saver()
                    
                # Run training and check training accuracy and validation accuracy
                with tf.Session() as sess:
                    iteration_counter += 1
                    print("")
                    print('Hyperparameter combination {}'.format(iteration_counter))
                    train_accuracy, val_accuracy, loss_history, \
                    train_acc_history, val_acc_history = train(sess, loss, accuracy, train_step, dr, 
                                                               X_train, y_train, X_val, y_val,
                                                               batch_size=bs, num_epoch=num_epoch, 
                                                               print_every=100, verbose=True)                    
                    print('learning rate={:.2e}, reg scale={:.2e}, '
                          'dropout rate={}, batch size={}, '
                          'train accuracy={:.3f}, validation accuracy={:.3f}'
                          .format(lr, rs, dr, bs, train_accuracy, val_accuracy))
                    hyperparameter_dict[(lr, rs, dr, bs)] = (train_accuracy, val_accuracy)
                    
                    # Save(overwrite) the model if its validation accuracy becomes the best
                    if val_accuracy > best_val_accuracy:
                        best_val_accuracy = val_accuracy
                        best_loss_history = loss_history
                        best_train_acc_history = train_acc_history
                        best_val_acc_history = val_acc_history
                        saver.save(sess, './' + model)
                        print('Best model so far. Model saved.')
            
# Sort and print result list 
print("")
print("")
print('Result list (sorted by validation accuracy)')
hyperparameter_dict = sorted(hyperparameter_dict.items(), key=lambda x:x[1][1])
for h in hyperparameter_dict:
    print('learning rate={:.2e}, reg scale={:.2e}, '
          'dropout rate={}, batch size={}, '
          'train accuracy={:.3f}, validation accuracy={:.3f}'
          .format(h[0][0], h[0][1], h[0][2], h[0][3], h[1][0], h[1][1]))
    
# Print loss history of best model
plt.plot(best_loss_history)
plt.grid(True)
plt.title('Loss history of the best model')
plt.xlabel('minibatch number')
plt.ylabel('minibatch loss')
plt.show()

# Print training accuracy history and validation accuracy history of best model
plt.plot(best_train_acc_history)
plt.plot(best_val_acc_history)
plt.grid(True)
plt.title('Training accuracy and Validation accuracy history of the best model')
plt.xlabel('Epoch number')
plt.ylabel('accuracy')
plt.show()

### Visualize model in TensorBoard
* Model graph can be checked below. 
* Tensor summaries can be check via command "tensorboard --logdir=path/to/log-directory". Once TensorBoard is running, navigate your web browser to localhost:6006 to view the TensorBoard.
* "tensorboard --logdir=path/to/log-directory --inspect" can be used to check issue when tensorboard does not work fine
* Check more details in https://www.tensorflow.org/get_started/summaries_and_tensorboard

In [ ]:
from IPython.display import display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def


def write_graph_html(max_const_size=32):
    """print TensorFlow graph"""
    save_model_path = './' + model
    loaded_graph = tf.Graph()
    
    with tf.Session(graph=loaded_graph) as session:
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(session, save_model_path)        
        graph_def = loaded_graph.as_graph_def()
        strip_def = strip_consts(graph_def, max_const_size=max_const_size)
        code = """
        <html><body>
            <script>
              function load() {{
                document.getElementById("{id}").pbtxt = {data};
              }}
            </script>
            <link 
              rel="import" 
              href="https://tensorboard.appspot.com/tf-graph-basic.build.html" 
              onload=load()
            >
            <div style="height:600px">
              <tf-graph-basic id="{id}"></tf-graph-basic>
            </div>
        </body></html>
        """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
        
        iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
        """.format(code.replace('"', '&quot;'))
        display(HTML(iframe))
        
        with open(save_model_path + '.html', 'w') as html:
            print(code, file=html)
            
        
write_graph_html()

### Evaluate model with Test set

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    test_accuracy = evaluate(X_test, y_test, accuracy)
    print("Test accuracy={:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

import os
from PIL import Image

X_new_raw = []
X_new = []
y_new = []

# Resize and normalize test images to 32x32
for image_file in os.listdir("image_from_web"):
    print(image_file)
    image = Image.open(os.path.join("image_from_web", image_file))
    image = image.resize((32, 32))
    image = np.array(image)
    X_new_raw.append(image)
    image = normalize(image, X_train_mean, X_train_std)
    X_new.append(image)
    y_new.append(int(image_file.split('_')[0]))

print(y_new)

# Show one traffic sign example of each class
num_row = 5
num_col = 1
size = 3

f, axarr = plt.subplots(num_col, num_row, figsize=(size * num_row, size * num_col))
counter = 0
for i in range(num_row):
    axarr[i].imshow(X_new_raw[i])
    axarr[i].set_title(str(y_new[i]) + ": "+ sign_names[y_new[i]])
    axarr[i].axis('off')

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

prediction = tf.argmax(logits, 1)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    model_prediction = sess.run(prediction,
                                feed_dict={x: X_new,
                                           y: y_new,
                                           is_training: False,
                                           dropout_rate: dropout_rates[0]})
    
    for idx, predict_class in enumerate(model_prediction):
        print("")
        print('Correct class = {}'.format(str(y_new[idx]) + ": " + sign_names[y_new[idx]]))
        print('predict_class = {}'.format(str(predict_class) + ": " + sign_names[predict_class]))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    model_accuracy = sess.run(accuracy,
                              feed_dict={x: X_new,
                                         y: y_new,
                                         is_training: False,
                                         dropout_rate: dropout_rates[0]})
    print('predition accuracy for new images is {:.3f}'.format(model_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

def get_top_5_softmax(sess, logits, X_data): 
    softmax = sess.run(tf.nn.softmax(logits), feed_dict={x: X_data, 
                                                         is_training: False, 
                                                         dropout_rate: dropout_rates[0]})    
    top_5_softmax = tf.nn.top_k(softmax, k=5)
    return sess.run(top_5_softmax)


with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    top_5 = get_top_5_softmax(sess, logits, X_new)
    print('')
    print('top_5.values {}'.format(top_5.values.shape))
    print(top_5.values)
    print('')
    print('top_5.indices {}'.format(top_5.indices.shape))
    print(top_5.indices)

### Show the top 5 softmax and corresponding labels for new images

In [ ]:
# Show the top 5 softmax and corresponding labels for some random traffic sign test images 
def show_top_5_softmax(top_5, img_index, X_show, y_show):
    num_display = len(img_index)    
    plt.figure(figsize=(20, 3 * num_display))
    for i in range(num_display):
        # Show image and class No. as image title
        plt.subplot(num_display, 2, 2*i+1)
        plt.imshow(X_show[img_index[i]]) 
        plt.title(str(y_show[img_index[i]]) + " - " + sign_names[y_show[img_index[i]]])
        plt.axis('off')
        # Show top 5 softmax and use corresponding sign name
        plt.subplot(num_display, 2, 2*i+2)
        plt.barh(np.arange(1, 6, 1), top_5.values[img_index[i], :])
        plt.yticks(np.arange(1, 6, 1), sign_names[top_5.indices[img_index[i]]])

    plt.show()

img_index = [0, 1, 2, 3, 4]
show_top_5_softmax(top_5, img_index, X_new_raw, y_new)

### Show the top 5 softmax and corresponding labels for some incorrectly recognized test images 

In [ ]:
# Get top 5 softmax for test images
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    top_5 = get_top_5_softmax(sess, logits, X_test)

# Filter out all incorrectly recognized test images
y_incorrect = []

for i in range(n_test):
    if y_test[i] != top_5.indices[i,0]:
        y_incorrect.append(i)

y_incorrect = np.array(y_incorrect)

# Show incorrectly recognized test images and its top 5 softmax
num_display = 10
random_index = np.random.randint(y_incorrect.shape[0], size=num_display)
img_index = y_incorrect[random_index] 

show_top_5_softmax(top_5, img_index, test['features'], y_test)


### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

x = tf.placeholder(tf.float32, (None, 32, 32, 3))    
conv2d = tf.layers.conv2d(inputs=x, filters=6, kernel_size=5, padding='VALID', activation=tf.nn.relu)
bn = tf.layers.batch_normalization(inputs=conv2d)
pool = tf.layers.max_pooling2d(bn, pool_size=2, strides=2, padding='SAME')
conv2d_1 = tf.layers.conv2d(inputs=pool, filters=16, kernel_size=5, padding='VALID', activation=tf.nn.relu)

with tf.Session() as sess: 
    # Show original test image
    plt.imshow(test['features'][0])
    plt.show()
    
    # Show normalized test image
    plt.imshow(X_test[0])
    plt.show()
    
    # Show some layers' activation of the test image
    sess.run(tf.global_variables_initializer())
    outputFeatureMap(X_test[:1], conv2d, activation_min=-1, activation_max=-1 ,plt_num=1)
    plt.show()
    outputFeatureMap(X_test[:1], bn, activation_min=-1, activation_max=-1 ,plt_num=1)
    plt.show()
    outputFeatureMap(X_test[:1], pool, activation_min=-1, activation_max=-1 ,plt_num=1)
    plt.show()
    outputFeatureMap(X_test[:1], conv2d_1, activation_min=-1, activation_max=-1 ,plt_num=1)
    plt.show()

from sklearn.utils import shuffle

def evaluate(X_data, y_data, accuracy, dropout_rate_value=0.4, batch_size=64):
    '''
    Evaluate predict accuracy on a data set, normally validation set
    
    Input:
    - X_data: data set for placeholder x
    - y_data: label set for placeholder y
    - accuracy: operator of prediction accuracy
    - dropout_rate_value: value of dropout rate
    - batch_size: mini-batch size for evaluation
    
    Return:
    - accuracy: total prediction accuracy on this data set
    '''
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        batch_accuracy = sess.run(accuracy,
                                  feed_dict={x: batch_x, 
                                             y: batch_y, 
                                             is_training: False,
                                             dropout_rate: dropout_rate_value})
        total_accuracy += (batch_accuracy * len(batch_x))
    return total_accuracy / num_examples


def train(sess, loss, accuracy, train_step, dropout_rate_value, X_train, y_train, X_val, y_val,
          batch_size=64, num_epoch=10, print_every=100, verbose=False):
    '''
    Run mini-batch training, show training loss, training accuracy, and validation accuracy

    Input:
    - sess: session to run model training
    - loss: operator of model loss
    - accuracy: operator of prediction accuracy
    - train_step: operator of training step to minimize loss
    - dropout_rate_value: value of dropout rate
    - X_train: training set data for placeholder x
    - y_train: training set label for place holder y
    - X_val: validation set data for placeholder x
    - y_val: validation set data for placeholder y
    - batch_size: mini-batch size of training data
    - num_epoch: number of training epochs
    - print_every: number of iterations to print training loss and accuracy
    - verbose: print detailed info when it set to True

    Returns:
    - train_accuracy: prediction accuracy on last epoch of training
    - validation_accuracy: prediction accuracy on validation set
    '''    
    
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)        
    
    train_accuracy_history = []
    val_accuracy_history = []
    loss_history = []
    
    # Define TensorBoard FileWriter for tensor visualization
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./' + model + '/train',sess.graph)
    
    # Define variables for sess.run()
    variables = [loss, accuracy, train_step, merged]
    
    if verbose:
        print("")
        print("Training...")
    
    for i in range(num_epoch):
        epoch_correction = 0.0
        X_train, y_train = shuffle(X_train, y_train)
        
        # Train model with mini batch of training data
        for offset in range(0, num_examples, batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            batch_x = batch_PCA(batch_x)   # Use PCA to do color augmentation
            batch_loss, batch_accuracy, \
            _, summary = sess.run(variables, 
                                  feed_dict={x: batch_x, 
                                             y: batch_y, 
                                             is_training: True,
                                             dropout_rate: dropout_rate_value})
            
            iter_counter = offset // batch_size
            
            # Add tensor summary to TensorBoard to visualize
            train_writer.add_summary(summary, iter_counter)  
            
            # Add batch loss to loss history
            loss_history.append(batch_loss)
            
            # Aggreate batch correct predictions to epoch level
            epoch_correction += batch_accuracy * len(batch_x)
        
            # Print loss and batch training accuracy after every "print_every" iterations            
            if verbose and iter_counter % print_every == 0:
                print("iteration:{}, batch loss={:.3f}, batch accuracy={:.3f}".
                      format(iter_counter, batch_loss, batch_accuracy))
                    
        # Add epoch training accuracy and epoch validation accuracy to accuracy history
        train_accuracy = epoch_correction / num_examples
        validation_accuracy = evaluate(X_val, y_val, accuracy, dropout_rate_value, batch_size)
        train_accuracy_history.append(train_accuracy)
        val_accuracy_history.append(validation_accuracy)
        
        # Print accuracy
        if verbose:
            print("EPOCH:{}, train_accuracy={:.3f}, validation accuracy={:.3f}".
                  format(i+1, train_accuracy, validation_accuracy))

    return train_accuracy, validation_accuracy, \
           loss_history, train_accuracy_history, val_accuracy_history